In [1]:
import pandas as pd
from datahandler import FileHandler, Preprocessor

## 개체명 인식 모델 학습용 데이터 생성 및 가공

In [3]:
docs = FileHandler.load_file('/Users/eunbin/Desktop/Projects/saessack-server/extra/data/kinnaver/dataset.txt')
print(type(docs))
print(docs[1])

데이터 개수 : 16680
<class 'list'>
저희집 여인초는 키가 왜 안 자랄까요,,. 안녕하세요! 지난 여름부터 키우기 시작한 여인초가 있는데요, 얘가 위로 자라진 않고 계속해서 새로운 잎들만 나오네요ㅠㅠ 이파리들도 계속해서 축축 처지는 것 같습니다...  1. 여인초마다 자라는 높이가 정해져 있는 건가요? 위로 성장을 안 해요 2. 이파리가 왤케 처질까요ㅠ 조언 부탁드립니다!! 좋은 하루 되세요!. 아무 이상 없이 정상적으로 자라고 있습니다. 자라는 높이가 정해져 있지는 않습니다.  이파리가 쳐지는 것 아닙니다. 너무너무 멋지게 잘 자라고 있습니다.. 잘 키우셨네요. 오래된 잎들을 보면 말려 있는 것이 평소 수분이 약간 부족하였을 것으로 보입니다. 화분에 물이 흡수되는 원리와 기본적인 실내식물 관리요령에 대한 제 블로그의 자료를 첨부합니다. 화분관리에 참고해서 건강하게 키우시기 바랍니다..


In [4]:
tag_path = '/Users/eunbin/Desktop/Projects/saessack-server/extra/data/tag/'

plant_csv = pd.read_csv(tag_path+'PLANT.csv', encoding='utf-8')
category_csv = pd.read_csv(tag_path+'CATEGORY.csv', encoding='utf-8')
disease_csv = pd.read_csv(tag_path+'DISEASE.csv', encoding='utf-8')
bug_csv = pd.read_csv(tag_path+'BUG.csv', encoding='utf-8')

def get_tags(csvfile, tag_type):
    tags = list(csvfile[tag_type])
    tags.sort(key=len)
    result = []
    for tag in tags:
        if tag not in result:
            result.append(tag)
    print(len(result))

    return result


In [5]:
plants = get_tags(plant_csv, 'PLT')
categories = get_tags(category_csv, 'CTG')
diseases = get_tags(disease_csv, 'DIS')
bugs = get_tags(bug_csv, 'BUG')
tags = {
    'PLT': plants,
    'CTG': categories,
    'DIS': diseases,
    'BUG': bugs
}

1813
16
15
27


### 전처리

In [6]:
# 불용어제거
docs = Preprocessor.remove_stopword(docs)

### BIO Tagging
[데이터 형식]
<br> 여인초 왜 안자라.
<br> sentences = [['여','인','초','왜','안','자','라','.']]
<br>ner_tags = [['B-PLT','I-PLT','I-PLT','O','O','O','O','O']]

In [36]:
import re

result = []
B = '-B'
I = '-I'
id = -1

for doc in docs:
    id += 1
    doc = doc.replace(' ', '')
    # 음절 배열
    sentences_x = []
    for syllable in doc: #syllable은 음절
        sentences_x.append(syllable) 

    # tag 배열
    ner_tags_x = []
    for x in range(len(doc)):
        ner_tags_x.append('O')

    # matching_tag = {
    #     'BUG' : [],
    #     'DIS' : [],
    #     'CTG' : [],
    #     'PLT' : [] 
    # }
    
    for tag_type in ['BUG', 'DIS', 'PLT', 'CTG']:
        for tag in tags[tag_type]:
            for match in re.finditer(tag, doc):
                # matching_tag[tag_type].append(tag) ### 매칭된 태그 이름 
                st = match.start()
                end = match.end()
                ner_tags_x[st] = tag_type+B
                if end-st != 0:
                    for i in range(st+1, end):
                        ner_tags_x[i] = tag_type+I

    # for match in re.finditer('\.', doc):
    #     st = match.start()
    #     ner_tags_x[st] = '[DOT]'

    dic = {
        'sentence' : ' '.join(sentences_x),
        'tagging' : ' '.join(ner_tags_x)
        # 'PLT' : list(set(matching_tag['PLT'])),
        # 'CTG' : list(set(matching_tag['CTG'])),
        # 'BUG' : list(set(matching_tag['BUG'])),
        # 'DIS' : list(set(matching_tag['DIS']))
        }
    result.append(dic)

In [38]:
import json
import random

random.shuffle(result)

with open('/Users/eunbin/Desktop/Projects/saessack-server/extra/data/kinnaver/sent_tag.json', 'w', encoding='utf-8') as jf:
    json.dump(result, jf, indent=2, ensure_ascii=False)

data = open('/Users/eunbin/Desktop/Projects/saessack-server/extra/data/kinnaver/sent_tag.json', encoding='utf-8')
df = pd.DataFrame(json.load(data))
df

,sentence,tagging
0,파 테 크 하 고 있 는 데 물 주 기 어 떻 게 하 나 요 . 대 파 를 심 었 ...,O O O O O O O O CTG-B CTG-I CTG-I O O O O O O ...
1,벤 자 민 가 지 치 기 . 벤 자 민 이 한 쪽 으 로 만 자 라 고 있 어 요 ...,O O O CTG-B CTG-I CTG-I CTG-I O O O O O O O O ...
2,먼 버 섯 이 에 요 . 먹 어 도 되 남 . 드 시 면 안 돼 죵 습 하 고 통 ...,O PLT-B PLT-I O O O O O O O O O O O O O O O O ...
3,다 육 이 질 문 이 다 육 이 이 름 이 랑 관 리 법 소 생 법 알 려 주 세 ...,PLT-B PLT-I O O O O PLT-B PLT-I O CTG-B CTG-I ...
4,선 인 장 도 와 주 세 요 . 최 근 에 물 을 2 주 간 격 으 로 줬 더 니 ...,PLT-B PLT-I PLT-I O O O O O O O O O O O O O O ...
...,...,...
16675,아 보 카 도 나 무 입 니 다 . 3 그 루 를 키 우 고 있 어 요 근 데 최 ...,PLT-B PLT-I PLT-I PLT-I O O O O O O O O O O O ...
16676,이 게 무 슨 식 물 인 가 요 . 키 워 보 라 고 주 셨 는 데 이 름 을 까 ...,O O O O O O O O O O O O O O O O O O O CTG-B CT...
16677,이 버 섯 이 뭔 가 요 . 오 빠 방 화 분 을 무 심 코 봤 는 데 첫 번 째 ...,O PLT-B PLT-I O O O O O O O O CTG-B CTG-I O O ...
16678,화 분 속 스 티 로 품 추 가 내 공 1 0 0 . 구 입 한 극 락 조 가 커 ...,CTG-B CTG-I O O O O O O O O O O O O O O O O PL...


In [39]:
# 배열 개수 다른거 있는지 확인
for i in range(len(sentences)):
    x = len(sentences[i])
    y = len(ner_tags[i])
    if x != y:
        print(i)

In [40]:
# 최대 길이 확인 -> 5728
maxlen = len(sentences[0])
cnt = -1
for x in sentences:
    cnt += 1
    if len(x) > maxlen:
        maxlen = len(x)
        print(cnt)


print(maxlen)

1
2
4
6
19
28
160
173
337
4639
9616
5728


In [14]:
total_len = 0
for x in sentences:
    total_len += len(x)

print(total_len / 16680)

273.9065347721822
